# English Fiction 2gram

In [1]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

sc=SparkContext()
spark = SparkSession.builder.getOrCreate()

In [2]:
# import text file
ngram = spark.sparkContext.sequenceFile('s3://datasets.elasticmapreduce/ngrams/books/20090715/eng-fiction-all/2gram/data')

In [3]:
a = ngram.first()

In [4]:
type(a)

tuple

In [5]:
ngram.take(5)

[(1, '! 00000\t1796\t1\t1\t1'),
 (2, '! 00000\t1832\t1\t1\t1'),
 (3, '! 00000\t1838\t1\t1\t1'),
 (4, '! 00000\t1840\t1\t1\t1'),
 (5, '! 00000\t1841\t1\t1\t1')]

### Pre-processing

In [6]:
import sys
import os,datetime, re
from pyspark.sql import Row
from pyspark.sql.types import *

In [7]:
def unpack_to_row(tp):
    (num, line) = tp
    string, y, n1, n2, n3 = re.split(r'[\t]', line)
    #return [num, string, int(y), int(n1), int(n2), int(n3)]
    return Row(index=num, two_gram=string, 
               year=int(y), 
               match_count=int(n1), page_count=int(n2), volume_count=int(n3))
               # year=datetime.datetime.strptime(y, '%Y')

In [8]:
rdd = ngram.map(lambda ls: unpack_to_row(ls))

In [9]:
rdd.take(5)

[Row(index=1, match_count=1, page_count=1, two_gram='! 00000', volume_count=1, year=1796),
 Row(index=2, match_count=1, page_count=1, two_gram='! 00000', volume_count=1, year=1832),
 Row(index=3, match_count=1, page_count=1, two_gram='! 00000', volume_count=1, year=1838),
 Row(index=4, match_count=1, page_count=1, two_gram='! 00000', volume_count=1, year=1840),
 Row(index=5, match_count=1, page_count=1, two_gram='! 00000', volume_count=1, year=1841)]

In [10]:
schema = StructType([
    StructField('index', IntegerType(), True),
    StructField('two_gram', StringType(), True),
    StructField('year', IntegerType(), True),
    StructField('match_count', IntegerType(), True),
    StructField('page_count', IntegerType(), True),
    StructField('volume_count', IntegerType(), True)
])

In [11]:
# Create data frame
df = spark.createDataFrame(rdd, schema)
print(df.schema)
df.show()

StructType(List(StructField(index,IntegerType,true),StructField(two_gram,StringType,true),StructField(year,IntegerType,true),StructField(match_count,IntegerType,true),StructField(page_count,IntegerType,true),StructField(volume_count,IntegerType,true)))
+-----+--------+----+-----------+----------+------------+
|index|two_gram|year|match_count|page_count|volume_count|
+-----+--------+----+-----------+----------+------------+
|    1| ! 00000|1796|          1|         1|           1|
|    2| ! 00000|1832|          1|         1|           1|
|    3| ! 00000|1838|          1|         1|           1|
|    4| ! 00000|1840|          1|         1|           1|
|    5| ! 00000|1841|          1|         1|           1|
|    6| ! 00000|1881|          1|         1|           1|
|    7| ! 00000|1883|          1|         1|           1|
|    8| ! 00000|1892|          2|         2|           2|
|    9| ! 00000|1894|          1|         1|           1|
|   10| ! 00000|1897|          1|         1|       

In [ ]:
df_sql = df.createOrReplaceTempView('df_sql')
df_sql = spark.sql("CACHE TABLE df_sql")

In [ ]:
df_count = spark.sql('select two_gram,sum(match_count) as count from df_sql\
                     group by two_gram\
                     order by count desc')
df_count.show()

In [ ]:
df_1000 = df_count.take(1000)
df_1000[:10]

In [ ]:
dicta = {}
for i in df_1000:
    dicta[i[0]]=i[1]    

In [53]:
!pip3 install wordcloud

     |████████████████████████████████| 364 kB 17.9 MB/s eta 0:00:01


In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

In [ ]:
image = Image.open('heart.jpg')
graph = np.array(image)
wc = WordCloud(background_color='white',mask=graph)
wc.generate_from_frequencies(dicta)
#wordcloud = WordCloud.fit_words(dicta)
plt.imshow(wc)
plt.axis('off')
plt.show()